In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import matplotlib.pyplot as plt 
%matplotlib inline
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

## 신조어

In [19]:
from ckonlpy.tag import Twitter
twitter = Twitter()

In [18]:
 len(add_words)

236

## 불용어

In [16]:
 # 불용어 (가사 빈도수 높은 + 감정분류와 무관한 단어 추가 중)
stop_w = ['all','이렇게','네가','있는','니가','없는','너의','너무','그런',
          'oh','whoo','tuesday','내가','너를','나를','we','this','the','그렇게',
          'so','am','baby','and','can','you','much','me','for','go','in',
          '은', '는', '이', '가', '하','부터','처럼','까지',
          'know','no','of','let','my','수','너','내','나','그','난','봐',
          '돼','건','모든','에서','에게','싶어','잖아',
          '날','널','수','것','못','말','넌','젠','하나','정말','알','여기',
          '다시','하게','니까',
          '때','아','더','게','또','채','일','걸','누구','나는','너는','라면',
          '같아','있어',
          '의','가','보','들','좀','잘','걍','과','도','를','으로','우린','하지',
          '해도','하고','없어','않아',
          '자','에','와','한','하다','네','있다','나의','해','다','내게','왜',
          '거야','이제','그냥','했던','하는']
# 우리 , 그대 , 사랑 , 지금

In [4]:
play_list = pd.read_csv('total_playlist_1206.csv')

In [7]:
play_list.drop('year',axis=1,inplace=True)

## Test 
* ['tags'].str.contains('봄')

In [8]:
play_list.head(1)

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
0,30392514,bath,offonoff (오프온오프),bath,5781,거긴 날씨가 어때 하늘과 밤엔 자꾸 떠오르는 기억이 계속 말을 건네네 우리는 함께일...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","['거긴', '날씨가', '어때', '하늘과', '밤엔', '자꾸', '떠오르는',..."


In [45]:
play_list[play_list['tags'].str.contains('봄')].head(1)

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
1323,6086049,이렇게 좋아해 본 적이 없어요,CHEEZE(치즈),이렇게 좋아해 본 적이 없어요 (소녀의 세계 X CHEEZE(치즈)),2185,따스한 바람이 불어오는 날엔 포근한 미소로 반겨주던 그대란 사람 설레는 맘 너에게 ...,https://image.bugsm.co.kr/album/images/200/403...,"['국내', '가요', '달달한', '봄', '봄캐롤', '연애권장송']","['따스한', '바람이', '불어오는', '날엔', '포근한', '미소로', '반겨..."


In [13]:
music_list = play_list[play_list['tags'].str.contains('봄')]

In [14]:
lyric = music_list.Lyric

In [15]:
lyric

1323     따스한 바람이 불어오는 날엔 포근한 미소로 반겨주던 그대란 사람 설레는 맘 너에게 ...
1324     우연히 오나 봄 향기가 보여 너도 같이 오나 저 멀리서 향기가 설레는 코 끝에 입술...
1325     어두운 불빛아래 촛불 와인 잔에 담긴 약속하나 항상 곁에서 지켜줄거야 믿어준 너였잖...
1326     커피를 많이 마셨나 봐요 심장이 막 두근대고 잠은 수가 없어요 한참 뒤에 별빛이 내...
1327     있잖아 언젠가 아주 어렸을 기억처럼 어쩌면 어쩌면 별이 노랠 할 있다고 믿어 손 잡...
                               ...                        
15114    혹시 너도 생각하는지 눈을 감아도 자꾸 보여 어쩌면 첨부터 예감했는지 몰라 소중한 ...
15115    언젠가 길을걷다가 들려오는 음악소리에 나도 모르게 너의모습이 잊혀진줄 알았었는데 오...
15116    어떤 옷을 입을까 머리는 어떻게 할까 만나는 날엔 행복한 병에 걸린 바보같아 이런게...
15117    두두 뚜두 두두 두두 뚜두 두두 늘 보고싶은 맘 니 볼에 입맞춤 하는 꿈     이...
15118    요즘 들어 부쩍 좋아져 나도 모르게 자꾸 보고 싶어져 마음을 모르는 건지 아니면 말...
Name: Lyric, Length: 1114, dtype: object

## TF-idf

In [65]:
# pattern = "(?u)\\b[\\w-]+\\b"

In [135]:
tfidf = TfidfVectorizer(stop_words=stop_w,
                        # token_pattern=pattern,
                        lowercase=True,
                        max_features=1000) # 1000개씩

In [67]:
tdm = tfidf.fit_transform(lyric.values.astype('U'))  ## Even astype(str) would work

In [24]:
words = tfidf.get_feature_names()
words[-5:]

['힘들어', '힘들어도', '힘들지', '힘을', '힘이']

In [25]:
tdm.indices

array([3522, 2081, 1209, ..., 1754, 2644, 2268], dtype=int32)

In [26]:
for i, n in sorted(zip(tdm[0].indices, tdm[0].data)):
    print(words[i], n)

건넨 0.07844801654632756
곁에서 0.184541224506875
곁을 0.06971270250431362
괜스레 0.08074570468514031
귓가에 0.12302748300458334
그날 0.06327507660111241
그대가 0.1378889138688838
그대란 0.07482253365091474
그대에게 0.12092586718664805
그대와 0.04881505852815969
그리고 0.17089232697801626
그저 0.19215712676576707
날엔 0.06532320422357225
내게는 0.06392149491931542
너무나 0.05518618087730147
너에게 0.03579659615509526
누군가를 0.07650534890538982
눈만 0.22001455619917937
다가가 0.0581652454545113
더는 0.06151374150229167
따스한 0.0539248896747608
뜨면 0.18798061115032424
말아요 0.059493040211111396
매일 0.036681337900855815
머물러줘요 0.2615499917650245
무심한 0.07482253365091474
문득 0.05903472082136191
미소로 0.06532320422357225
바랄 0.07844801654632756
바람이 0.047668596313676606
보고 0.1091384091418456
보며 0.04661778840470897
부디 0.07844801654632756
불어오는 0.06327507660111241
사람 0.04350676037827177
사랑스러운 0.13380111899305047
사실 0.047668596313676606
설레는 0.05123334466142465
설렘이 0.06870397274545255
손잡아 0.25067354307878603
순간은 0.07650534890538982
싶어요 0.182932165386899
아름다워요 

In [27]:
count = tdm.sum(axis=0)

In [28]:
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    '빈도': count.flat})


In [31]:
display(word_count.head(),word_count.tail())

,단어,빈도
0,가게,0.569420
1,가고,3.070720
2,가기,0.898813
3,가까운,0.759125
4,가까운데,0.429230


,단어,빈도
4995,힘들어,1.503586
4996,힘들어도,0.655391
4997,힘들지,0.518327
4998,힘을,1.495998
4999,힘이,2.592416


In [30]:
len(word_count)

5000

In [32]:
sorted_df = word_count.sort_values('빈도', ascending=False)  # 내림차순 정렬
sorted_df.head(10)

,단어,빈도
926,너와,22.602717
4221,좋아,22.565149
4795,함께,17.231941
919,너에게,15.634214
143,같은,14.388375
1754,매일,13.823037
3847,이런,13.737105
3898,이젠,13.497196
2633,사랑해,13.280414
2268,보고,13.215661


In [41]:
pd.concat(result)

,단어,빈도,category
926,너와,22.602717,봄
4221,좋아,22.565149,봄
4795,함께,17.231941,봄
919,너에게,15.634214,봄
143,같은,14.388375,봄
...,...,...,...
661,끝으로,0.183061,여름
656,끝없는,0.183061,여름
2948,시선,0.160723,여름
65,가잖아,0.160723,여름


---

## 카테고리별 단어 빈도수
'신나는','발라드한','그루브한', '감성적인','어쿠스틱한','일렉트로닉','달달한','몽환적인',
        '올디스','강한','잔잔한','애절한','감각적인','섹시한','쓸쓸한','소울풀한',
        '청량한','봄','여름','가을','겨울','아침','오후','저녁','밤/새벽','화창한 날',
        '비/흐림','눈오는 날','비온후/맑게갠','선선한','쌀쌀한','드라이브','운동/헬스',
        '하교/퇴근길','등교/출근길','클럽/파티','카페','노래방','산책/여행','사무실',
        '편집숍/매장','잠들기 전','결혼','페스티벌','혼술혼밥','사랑/기쁨' ,'이별/슬픔',
        '스트레스/짜증','우울할때','지치고 힘들때','멘붕/불안','그리움','외로울때','썸 탈때',
        '고백','울고 싶을때','새벽감성','싱숭생숭','설렘/심쿵','기분전환','힐링'

In [156]:
test = ['신나는','발라드한','그루브한', '감성적인','어쿠스틱한','일렉트로닉','달달한','몽환적인',
        '올디스','강한','잔잔한','애절한','감각적인','섹시한','쓸쓸한','소울풀한',
        '청량한','봄','여름','가을','겨울','아침','오후','저녁','밤/새벽','화창한 날',
        '비/흐림','눈오는 날','비온후/맑게갠','선선한','쌀쌀한','드라이브','운동/헬스',
        '하교/퇴근길','등교/출근길','클럽/파티','카페','노래방','산책/여행','사무실',
        '편집숍/매장','잠들기 전','결혼','페스티벌','혼술혼밥','사랑/기쁨' ,'이별/슬픔',
        '스트레스/짜증','우울할때','지치고 힘들때','멘붕/불안','그리움','외로울때','썸 탈때',
        '고백','울고 싶을때','새벽감성','싱숭생숭','설렘/심쿵','기분전환','힐링']

In [157]:
len(test)

61

In [192]:
result =[]
for i in tqdm(test):
    music_list = play_list[play_list['tags'].str.contains(i)] # 리스트에 담긴 단어가 포함된 태그의 노래 목록을 가져옴
    lyric = music_list.Lyric # 가사만 저장
    tdm = tfidf.fit_transform(lyric.values.astype('str')) # TF-idf 적용
   #tdm = tfidf.fit_transform(lyric.values.astype('U'))  ## Even astype(str) would work
    count = tdm.sum(axis=0) # 단어 count 
    word_count = pd.DataFrame({
        '단어': tfidf.get_feature_names(),
        '빈도': count.flat})                     # DataFrame 생성
    sorted_df = word_count.sort_values('빈도', ascending=False)  # 내림차순 정렬
    sorted_df['category'] = i # 카테고리 명 (현재 반복문에서 입력된 단어)
    result.append(sorted_df)


100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:07<00:00,  7.97it/s]


In [193]:
cc =pd.concat(result)

In [194]:
cc.groupby('category')['category'].count()

category
가을        1000
감각적인      1000
감성적인      1000
강한        1000
겨울        1000
          ... 
편집숍/매장    1000
하교/퇴근길    1000
혼술혼밥      1000
화창한 날     1000
힐링        1000
Name: category, Length: 61, dtype: int64

In [196]:
dd.to_csv('word_freq_category.csv',index=False)

## 연대별 단어 빈도수 
* 1940 ~2010 년대 + 2020 + 2021

In [198]:
years = pd.read_csv('sort_total_years.csv')

In [201]:
year_list = years.year.unique().tolist()

In [208]:
result =[]
for i in tqdm(year_list):
    music_list = years[years['year'].str.contains(i)]
    lyric = music_list.Lyric
    tdm = tfidf.fit_transform(lyric.values.astype('str'))
   #tdm = tfidf.fit_transform(lyric.values.astype('U'))  ## Even astype(str) would work
    count = tdm.sum(axis=0)
    word_count = pd.DataFrame({
        '단어': tfidf.get_feature_names(),
        '빈도': count.flat})
    sorted_df = word_count.sort_values('빈도', ascending=False)  # 내림차순 정렬
    sorted_df['category'] = i # 카테고리 명
    result.append(sorted_df)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.05it/s]


In [209]:
yc =pd.concat(result)

In [210]:
yc.groupby('category')['category'].count()

category
1940y    1000
1950y    1000
1960y    1000
1970y    1000
1980y    1000
1990y    1000
2000y    1000
2010y    1000
2020y    1000
2021y    1000
Name: category, dtype: int64

In [211]:
yc.to_csv('word_freq_years.csv',index=False)

---

In [ ]:
### XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXxXXXXXXXXXXxXXXXXXXXXXxXXXXXXXXXXx

## Error
Python Pandas ValueError Arrays Must be All Same Length
> https://stackoverflow.com/questions/40442014/python-pandas-valueerror-arrays-must-be-all-same-length

In [ ]:
'''a = {'Links' : lines ,'Titles' : titles , 'Singers': finalsingers , 'Albums':finalalbums , 'Years' : years}
df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()'''

In [50]:
from tqdm import tqdm

### 저장
* joblib은 pickle과 비슷하게 파이썬의 값들을 저장해주는 라이브러리
* pickle에 비해서 수치 데이터를 더 효율적으로 저장
* tfidf, tdm, 그리고 df 의 'sentiment' 열을 피클로 저장 (향후에 감정분석에서 사용 가능)

In [ ]:
from sklearn.externals import joblib

In [ ]:
with open('amazon.pkl', 'wb') as f:
    joblib.dump(
        {'vectorizer': tfidf, 'tdm': tdm, 'sentiment': sentiment},
        f
    )

## Error
ValueError: empty vocabulary; perhaps the documents only contain stop words
AttributeError: 'numpy.ndarray' object has no attribute 'split'